In [ ]:
%pylab inline

In [ ]:
from seq_draw import rf_pulses, atoms, gradients, misc, overlays, SeqDiagram, seq_blocks

In [ ]:

# init sequence diagram
sd = SeqDiagram()

# inti with useful start values
sd.init_axes()
print sd.sqaxes.keys()

# internally axes are dictionarys which store elemnts in a list (key=atoms)
print sd.sqaxes['rf']

# define draw style
std_kw = {'linestyle': 'solid', 'color': 'black'}
rf_kw = {'linestyle': 'solid', 'color': 'black'}
grad_kw = {'linestyle': 'solid', 'color': 'black'}
vline_kw = {'linestyle': 'dashed', 'color': 'gray', 'linewidth': 1.}

# define timings
start_delay = 0.75
echo_time = 5.0
nref = 1
rf_pulse_dur = 0.5
rf_intensity = 0.1
grad_read_dur = 0.1
grad_read_int = 0.025
grad_blip_dur = 0.05
crush_dur = 0.1
crush_int = 3*grad_read_int
enc_dur = grad_read_dur
enc_int = 0.05
nav_rep = 3
epi_rep = 13

# define rf pulses
exc = rf_pulses.SincPulse(sd, 'rf', rf_pulse_dur, rf_intensity, label=u'90˚', side_lobes=10, plot_kw=rf_kw)
ref = rf_pulses.SincPulse(sd, 'rf', rf_pulse_dur, 1.5*rf_intensity, label=u'180˚', label_coord=(-0.125, 0.15), plot_kw=rf_kw)

# gradients
slab_sel = gradients.RectGrad(sd, 'gz', rf_pulse_dur, grad_read_int, plot_kw=grad_kw)
slab_sel_reph = gradients.RectGrad(sd, 'gz', rf_pulse_dur/2, -grad_read_int*2, plot_kw=grad_kw)
crush_grad = gradients.RectGrad(sd, 'gz', crush_dur, crush_int, plot_kw=grad_kw)
read_enc = gradients.RectGrad(sd, 'gx', enc_dur, grad_read_int * 0.5)
pe1_enc = gradients.RectGrad(sd, 'gy', enc_dur, -grad_read_int)
pe2_enc = gradients.TrapGradTable(sd, 'g2', enc_dur, enc_int, -enc_int, tab_type=gradients.GradTableType.CENTER_OUT,
                                  ft_ramp_ratio=1e6, plot_kw=grad_kw)

# navigators
pc_scans = seq_blocks.get_epi_readout(sd, 'gy', nav_rep, grad_read_dur, grad_read_int, read_grad=slab_sel)

# EPI readout
epi_read = seq_blocks.get_epi_readout(sd, 'gx', epi_rep, grad_read_dur, grad_read_int, read_grad=slab_sel)
blip = gradients.BlipGrad(sd, 'gy', grad_blip_dur, grad_read_int, plot_kw=grad_kw)
epi_pe1 = seq_blocks.get_epi_blips(sd, 'gy', epi_rep - 1, grad_blip_dur, grad_read_int, grad_read_dur, blip_grad=blip)
epi_pe2 = seq_blocks.get_epi_blips(sd, 'gz', epi_rep - 1, grad_blip_dur, grad_read_int, grad_read_dur, blip_pattern=[1., 1., -2.], blip_grad=blip)
epi_duration = sd.get_total_duration(epi_read)
refocusing_duration = sd.get_total_duration([crush_grad, ref, crush_grad])
echo_time = 2. * (epi_duration/2 + refocusing_duration/2 + enc_dur)
print epi_duration, refocusing_duration

# define some time spans
delta = misc.Line(sd, 'rf', 0.5, plot_kw=std_kw)
delta_crush = misc.Line(sd, 'rf', crush_dur, plot_kw=std_kw)

## Add pulses to axes
# Start with excitation and navigators
sd.fill(tp=start_delay)
sd.add_atom(sd.set_axis([exc], 'rf'))
sd.add_atom(sd.set_axis([delta], 'gx'))
sd.add_atom(sd.set_axis([delta, pc_scans], 'gy'))
sd.add_atom(sd.set_axis([slab_sel, slab_sel_reph], 'gz'))
sd.fill(axes=['rf','gx', 'gy', 'gz'], tp=start_delay + rf_pulse_dur/2 + echo_time/2 - crush_dur - rf_pulse_dur/2)
# refocusing and EPI trajectory
sd.add_atom(sd.set_axis([delta_crush, ref], 'rf'))
sd.add_atom(sd.set_axis([crush_grad, delta, crush_grad, read_enc, epi_read,read_enc], 'gx'))
sd.add_atom(sd.set_axis([crush_grad, delta, crush_grad, pe1_enc, epi_pe1,pe1_enc], 'gy'))
sd.add_atom(sd.set_axis([crush_grad, slab_sel, crush_grad, pe2_enc, epi_pe2,pe2_enc], 'gz'))
sd.fill()
# 2nd refocusing
sd.add_atom(sd.set_axis([delta_crush, ref], 'rf'))
sd.add_atom(sd.set_axis([crush_grad, delta, crush_grad], 'gx'))
sd.add_atom(sd.set_axis([crush_grad, delta, crush_grad], 'gy'))
sd.add_atom(sd.set_axis([crush_grad, slab_sel, crush_grad], 'gz'))
sd.fill()

# add echo time arrow
te_span = overlays.TimeSpan(sd, start_delay + rf_pulse_dur/2, echo_time, .975, 'TE')
te_span.draw()
vline1 = overlays.VerticalLine(sd, start_delay + rf_pulse_dur/2, plot_kw=vline_kw, ymax=0.975)
vline1.draw()
vline2 = overlays.VerticalLine(sd, start_delay + rf_pulse_dur/2 + echo_time, plot_kw=vline_kw, ymax=0.975)
vline2.draw()

# draw
sd.draw(debug=False)

# save
fig = gcf()
fig.savefig('3DGRASE.pdf', bbox_inches='tight')

In [ ]:
# SeqDiagram shows all axes with its elements when printed
sd